原模型

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb

# 參數設定
max_features = 20000
maxlen = 80
batch_size = 32
embedding_size = 128
lstm_units = 128
epochs = 10

# 加載IMDB數據集
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 將序列數據填充到固定長度
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# 構建LSTM模型
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# 編譯模型
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# 評估模型
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

17464789/17464789 [==============================] - 0s 0us/step
Epoch 1/10
782/782 [==============================] - 257s 324ms/step - loss: 0.4394 - accuracy: 0.7860 - val_loss: 0.3701 - val_accuracy: 0.8357
Epoch 2/10
782/782 [==============================] - 278s 356ms/step - loss: 0.2597 - accuracy: 0.8971 - val_loss: 0.4211 - val_accuracy: 0.8197
Epoch 3/10
782/782 [==============================] - 262s 336ms/step - loss: 0.1679 - accuracy: 0.9364 - val_loss: 0.4176 - val_accuracy: 0.8299
Epoch 4/10
275/782 [=========>....................] - ETA: 2:22 - loss: 0.0916 - accuracy: 0.9698

**說明：**

數據集：這裡使用了Reuters新聞數據集，你可以將其替換為自己的數據集。

預訓練詞嵌入：使用GloVe詞嵌入來初始化嵌入層，這通常能夠提高模型的性能。

雙層LSTM：增加了LSTM層的數量以捕捉更多特徵。

Dropout：在LSTM層和全連接層中使用Dropout來防止過擬合。

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant

# 加載數據集 (這裡使用一個示例數據集，你可以替換成自己的數據集)
from keras.datasets import reuters
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=20000)

# 參數設定
max_features = 20000
maxlen = 100
embedding_dim = 100
batch_size = 32
lstm_units = 128
epochs = 10

# 將序列數據填充到固定長度
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# 構建詞嵌入索引 (使用GloVe)
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# 構建詞嵌入矩陣
tokenizer = Tokenizer(num_words=max_features)
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# 構建LSTM模型
model = Sequential()
embedding_layer = Embedding(max_features, embedding_dim, embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen, trainable=False)
model.add(embedding_layer)
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 編譯模型
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# 評估模型
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)